## Description:

This notebook uses manual interpolation as well as a grid of points spaced "PIXELS_PER_POINT" apart to re-interpolate a Moving image onto a Reference. The code currently assumes that the reference and moving image share exact dimensions along all 3 axes, e.g. as would be the case if these samples were previously aligned using the Elastix pipeline.

## Imports

In [1]:
import subprocess, numpy as np, tifffile, matplotlib.pyplot as plt, os
from skimage.measure import label, regionprops
import joblib as jl
from tqdm.notebook import tqdm

import glob, os, re, shutil, subprocess, time, traceback, tifffile, numpy as np, numba as nb, \
    scipy.stats, joblib as jl # pgzip
from itertools import chain
from skimage.measure import label, regionprops

## Specify files & alignment parameters

In [2]:
fnamePtsMov = 'Z:\\StandardBrain\\data\\BRAIN004rA_annotatedResultOfFitTo19w-23.3.10\\pointlabels.tif'
fnamePtsRef = 'Z:\\StandardBrain\\data\\BRAIN019wRAR_reannotatedReferenceForFitOfBrain4-23.3.10\\pointlabels.tif'

In [3]:
# Set SUBSAMPLE to 2 or 4 to speed up testing
SUBSAMPLE = 2

In [4]:
# Pixel adjustment should be a function of SUBSAMPLE.
# For an image of ~1000 px/axis, and SUBSAMPLE=1, PIXELS_PER_POINT=100 
# leads to 10 control points along each axis (~10^3 total)
PIXELS_PER_POINT = 100 / SUBSAMPLE

In [5]:
# Kernel from: https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RBFInterpolator.html
INTERPOLATION_KERNEL = 'thin_plate_spline'

In [6]:
# Output directory
OUTPUT_DIRECTORY = 'Z:/StandardBrain/manualpolish/BRAIN019wRAR_BRAIN004rA/'

## Obtain corresponding annotated points

In [7]:
imgPtsMov = tifffile.imread(fnamePtsMov)
imgPtsRef = tifffile.imread(fnamePtsRef)

In [8]:
imgMov = tifffile.imread(fnamePtsMov.replace('pointlabels.tif', 'synapsin.tif'))
imgRef = tifffile.imread(fnamePtsRef.replace('pointlabels.tif', 'synapsin.tif'))

In [9]:
# This code assumes MOV/REF are already the same exact pixel size
assert imgMov.shape == imgRef.shape
assert imgPtsMov.shape == imgPtsRef.shape
assert imgMov.shape == imgPtsMov.shape

In [10]:
# Do tests at lower resolution
imgPtsMov = imgPtsMov[::SUBSAMPLE, ::SUBSAMPLE, ::SUBSAMPLE]
imgPtsRef = imgPtsRef[::SUBSAMPLE, ::SUBSAMPLE, ::SUBSAMPLE]

imgMov = imgMov[::SUBSAMPLE, ::SUBSAMPLE, ::SUBSAMPLE]
imgRef = imgRef[::SUBSAMPLE, ::SUBSAMPLE, ::SUBSAMPLE]

In [11]:
imgPtsMov.shape, imgPtsRef.shape, imgMov.shape, imgRef.shape

((455, 523, 523), (455, 523, 523), (455, 523, 523), (455, 523, 523))

## Generate grid of points at least R distance away

In [12]:
# Obtain shared points
idsMov = np.unique(imgPtsMov)
idsRef = np.unique(imgPtsRef)

idsShared = np.intersect1d(idsMov, idsRef)
idsShared = idsShared[idsShared != 0]
idsShared

array([1, 2, 3, 4, 5, 6], dtype=int32)

In [13]:
ptsMov = np.array([np.mean(np.argwhere(imgPtsMov == x), axis=0) for x in idsShared])
ptsRef = np.array([np.mean(np.argwhere(imgPtsRef == x), axis=0) for x in idsShared])
ptsMov.shape

(6, 3)

In [14]:
gridMov = np.array(np.meshgrid(
    np.linspace(0, imgPtsMov.shape[0], int(np.ceil(imgPtsMov.shape[0] / PIXELS_PER_POINT))),
    np.linspace(0, imgPtsMov.shape[1], int(np.ceil(imgPtsMov.shape[1] / PIXELS_PER_POINT))),
    np.linspace(0, imgPtsMov.shape[2], int(np.ceil(imgPtsMov.shape[2] / PIXELS_PER_POINT))))).reshape((3, -1)).T

gridRef = np.array(np.meshgrid(
    np.linspace(0, imgPtsRef.shape[0], int(np.ceil(imgPtsRef.shape[0] / PIXELS_PER_POINT))),
    np.linspace(0, imgPtsRef.shape[1], int(np.ceil(imgPtsRef.shape[1] / PIXELS_PER_POINT))),
    np.linspace(0, imgPtsRef.shape[2], int(np.ceil(imgPtsRef.shape[2] / PIXELS_PER_POINT))))).reshape((3, -1)).T

In [15]:
# Remove points from this grid that are too close to either ref/mov manual annotations
toocloseM = np.min(np.linalg.norm(gridMov[:, np.newaxis, :] - ptsMov[np.newaxis, :, :], axis=2), axis=1) > PIXELS_PER_POINT
toocloseR = np.min(np.linalg.norm(gridRef[:, np.newaxis, :] - ptsRef[np.newaxis, :, :], axis=2), axis=1) > PIXELS_PER_POINT

gridNotTooCloseMov = gridMov[toocloseM | toocloseR]
gridNotTooCloseRef = gridRef[toocloseM | toocloseR]

In [16]:
# Concatenate grid points with manual points
alignMov = np.vstack((gridNotTooCloseMov, ptsMov))
alignRef = np.vstack((gridNotTooCloseRef, ptsRef))

## Run cubic spline regression to correct for deformation

In [17]:
# Build interpolation from REF --> MOV
interp = scipy.interpolate.RBFInterpolator(alignRef, alignMov, kernel=INTERPOLATION_KERNEL)

In [18]:
# Transform REF coordinates into MOV coordinates
def interpolatePlane(z, shp):
    # Get ref coords
    xyzRef = np.array(np.meshgrid(
        np.arange(z, z+1),
        np.arange(shp[1]),
        np.arange(shp[2]))).reshape(3, -1).T
    # Interpolate
    xyzMov = interp(xyzRef)
    xyzMov[:, 0] = np.clip(xyzMov[:, 0], 0, shp[0]-1)
    xyzMov[:, 1] = np.clip(xyzMov[:, 1], 0, shp[1]-1)
    xyzMov[:, 2] = np.clip(xyzMov[:, 2], 0, shp[2]-1)
    # Done!
    return np.hstack((xyzRef, xyzMov))

In [19]:
xyzMov = np.vstack(jl.Parallel(n_jobs=58)(jl.delayed(
    interpolatePlane)(z, imgPtsRef.shape) for z in tqdm(range(imgPtsRef.shape[0]))))
xyzMov.shape

  0%|          | 0/455 [00:00<?, ?it/s]

(124455695, 6)

## Resample image according to interpolated coordinates

In [20]:
# Reconstruct reference as a sanity check (this image should match the original input)
imgAlignedRef = scipy.ndimage.map_coordinates(imgRef, xyzMov[:, 0:3].T, order=2).reshape(imgRef.shape)
imgAlignedRef.shape

(455, 523, 523)

In [21]:
# Reconstruct moving image into the aligned coordinate system of the reference
imgAlignedMov = scipy.ndimage.map_coordinates(imgMov, xyzMov[:, 3:6].T, order=2).reshape(imgRef.shape)
imgAlignedMov.shape

(455, 523, 523)

In [22]:
# Reconstruct moving image in its original coordinates
imgOrgMov = scipy.ndimage.map_coordinates(imgMov, xyzMov[:, 0:3].T, order=2).reshape(imgRef.shape)
imgOrgMov.shape

(455, 523, 523)

## Store output

In [23]:
os.makedirs(OUTPUT_DIRECTORY, exist_ok=True)

In [24]:
tifffile.imwrite(os.path.join(OUTPUT_DIRECTORY, 'ref_resampled.tif'), imgAlignedRef, bigtiff=True)
tifffile.imwrite(os.path.join(OUTPUT_DIRECTORY, 'mov_resampled.tif'), imgAlignedMov, bigtiff=True)
tifffile.imwrite(os.path.join(OUTPUT_DIRECTORY, 'mov_original.tif'), imgOrgMov, bigtiff=True)